This document is used to analysis all failed cases executed by previous [functional test](./capability.ipynb).

In [1]:
from collections import Counter
from enum import IntEnum
from pathlib import Path
import pickle
from pprint import pprint
import shutil
import subprocess
from tqdm import tqdm
from typing import Dict, List, Optional

# Preparation

In [2]:
IWASM = Path(shutil.which("iwasm.dbg"))
ADDR2LINE = Path(shutil.which("addr2line"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK.exists(), "use `git submodule update` to fetch cookbook"

WASI_SDK_HOME = Path("/opt/wasi-sdk-22.0")

WABT_HOME = Path("/opt/wabt-1.0.35")


In [3]:
# sync with [functional test](./capability.ipynb)
class CaseErrorCode(IntEnum):
    OK = 0
    COMPILATION_FAILURE = 1
    WASM_EXECUTION_FAILURE = 2
    PY_EXECUTION_FAILURE = 3
    DIFFERENT_RESULT = 4
    BYPASS = 5
    WASM_EXECUTION_TIMEOUT = 6
    PY_EXECUTION_TIMEOUT = 7

    def __str__(self):
        return f'{self.name}'

class CaseResult:
    def __init__(self, error_code: CaseErrorCode, msg: str):
        self.error_code = error_code
        self.msg = msg

    def __repr__(self):
        return f"{self.error_code}. \"{self.msg.strip()}\""


In [4]:
def restore_result(out_file: Path) -> Dict[str, CaseResult]:
    with out_file.open("rb") as f:
        return pickle.load(f)

def filter_wasm_execution_failure(reulsts: Dict[str, CaseResult]) -> Dict[str, CaseResult]:
    return {k: v for k, v in reulsts.items() if v.error_code == CaseErrorCode.WASM_EXECUTION_FAILURE}

def filter_result_error_msg(result: CaseResult, keyword: str) -> bool:
    for line in result.msg.split('\n'):
        # print(f"--> {line}")
        if line.startswith(keyword):
            return True
    
    return False

def case_name_to_wasm_file(chapter_n: str, case_name: str) -> Path:
    """
    if chapter_n is 0, and case_name is logging_test_output_to_a_file.test.
    the output should be
    0/logging_test_output_to_a_file/test.wasm
    """
    return PYTHON_COOKBOOK.joinpath(f"{chapter_n}/{case_name.replace('.', '/')}.wasm")

# Analysis

In [5]:
# {chapter number : {case_name, CaseResult}}
chapter_x_result = {}
for i in range(1, 15):
    chapter_x_result[i] = restore_result(Path(f"chapter_{i}_result.pkl"))

## `WASM_EXECUTION_FAILURE`

It seems all failure can be separated into two kinds according to their error message
- Exception
- Traceback

In [6]:
# {chapter number : {case_name: CaseResult}}
chapter_x_wasm_exec_failed = {}
for i in range(1, 15):
    chapter_x_wasm_exec_failed[i] = filter_wasm_execution_failure(chapter_x_result[i])

In [7]:
# {case_name: CaseResult}
chapter_x_wamr_exception = {}
chapter_x_wasm_python_exception = {}

counter = []
for chapter_n, chapter_i_wasm_exec_failed in chapter_x_wasm_exec_failed.items():
    for case_name, result in chapter_i_wasm_exec_failed.items():
        wasm_file = case_name_to_wasm_file(chapter_n, case_name)
        assert wasm_file.exists()

        if filter_result_error_msg(result, "Exception: "):
            chapter_x_wamr_exception[case_name] = result
            counter.append("Exception")

        elif filter_result_error_msg(result, "Traceback "):
            chapter_x_wasm_python_exception[case_name] =result
            counter.append("Traceback")

        else:
            print(f"chapter {i} {case_name}: {result}")
            assert False, f"unexpected failure message.  chapter {i} {case_name}: {result}"

counter = Counter(counter)
print(counter)

Counter({'Traceback': 18, 'Exception': 6})


### Exception(throw by wamr)

List all exception messages thrown by wamr. It turns out that all exceptions are thrown by WAMR can be splited into two kinds:
- *indirect call type mismatch*
- *out of bounds memory access*

In [8]:
counter = []
for case_name, result in chapter_x_wamr_exception.items():
    for line in result.msg.split("\n"):
        if line.startswith("Exception: "):
            counter.append(line)
            break

pprint(Counter(counter))

Counter({'Exception: out of bounds memory access': 4,
         'Exception: indirect call type mismatch': 2})


Use `addr2line` and a debug building of `iwasm` to find out the exact line of (runtime) code that throws the exception.

> ⚠️ iwasm should be compiled with `-DCMAKE_BUILD_TYPE=Debug -DWAMR_BUILD_DUMP_CALL_STACK=1 -DWAMR_BUILD_FAST_INTERP=0 -DWAMR_BUILD_CUSTOM_NAME_SECTION=1 -DWAMR_BUILD_LOAD_CUSTOM_SECTION=1`

In [9]:
# traverse all cases in `chapter_x_wamr_exception`
# run the case with iwasm and export the callstack. `iwasm xxx.wasm > xxx.callstack`
# use addr2line to analyze the callstack. 
#   `addr2line --wasi-sdk {wasi_sdk_home} --wabt {wabt_home} --wasm-file xxx.wasm xxx.callstack`
# and save the result to `xxx.detail.callstack`

def execute_iwasm(wasm_file: Path, out_file: Path) -> subprocess.CompletedProcess:
    with open(out_file, "wt") as f:
        return subprocess.run(
            [
                IWASM, 
                "--dir=.",
                "--stack-size=134217728",
                wasm_file
            ],
            stdout=f,
            stderr=subprocess.PIPE,
            check=False,
        )

def addr2line(wasm_file: Path, callstack_file: Path, out_file: Path) -> subprocess.CompletedProcess:
    with open(out_file, "wt") as f:
        return subprocess.run(
            [
                ADDR2LINE, 
                "--wasi-sdk", WASI_SDK_HOME, 
                "--wabt", WABT_HOME, 
                "--wasm-file", wasm_file, 
                callstack_file
            ],
            stdout=f,
            stderr=subprocess.PIPE,
            check=True,
        )

def output_top_N_callstack_detail(callstack_detail_file: Path, N: int) -> List[str]:
    top_n_bt = []
    with open(callstack_detail_file) as f:
        for line in f.readlines():
            if "at " in line:
                continue
            
            top_n_bt.append(line.strip())
            N -= 1

            if N == 0:
                break
    
    return top_n_bt

def analyze_callstack(wasm_file: Path, topN: int) -> List[str]:
    callstack_file = wasm_file.with_suffix(".ctk")
    callstack_detail_file = wasm_file.with_suffix(".dtl.ctk")

    execute_iwasm(wasm_file, callstack_file)
    addr2line(wasm_file, callstack_file, callstack_detail_file)
    top_n_bt = output_top_N_callstack_detail(callstack_detail_file, topN)

    return top_n_bt

def compare_callstack(ctk_1: List[str], ctk_2: List[str]) -> bool:
    if len(ctk_1) != len(ctk_2):
        return False
    
    return all([c1 == c2 for c1, c2 in zip(ctk_1, ctk_2)])

def summarize_callstacks(chapter_x_wasm_exec_failed, error_msg_key: str, pkl_name: str):
    variant_call_stacks = []

    for chapter_n, chapter_i_wasm_exec_failed in tqdm(chapter_x_wasm_exec_failed.items()):
        for case_name, result in chapter_i_wasm_exec_failed.items():
            if not filter_result_error_msg(result, error_msg_key):
                continue

            print(f"chapter {chapter_n} {case_name}: {result}")

            wasm_file = case_name_to_wasm_file(chapter_n, case_name)
            assert wasm_file.exists()

            wasm_file = case_name_to_wasm_file(chapter_n, case_name)
            assert wasm_file.exists(), f"{wasm_file} doesn't exist"

            top_n_bt = analyze_callstack(wasm_file, 5)
            for item in variant_call_stacks:
                if compare_callstack(item, top_n_bt):
                    break
            else:
                variant_call_stacks.append(top_n_bt)

    print(f"✨ There are {len(variant_call_stacks)} different callstacks.")
    pprint(variant_call_stacks)

    with open(pkl_name, "wb") as f:
        pickle.dump(variant_call_stacks, f)

#### indirect call type mismatch


In [11]:
summarize_callstacks(
    chapter_x_wasm_exec_failed,
    "Exception: indirect call type mismatch", 
    "indirect_call_type_mismatch_call_stack.pkl"
)

  0%|          | 0/14 [00:00<?, ?it/s]

chapter 14 logging_test_output_to_a_file.test: WASM_EXECUTION_FAILURE. "Exception: indirect call type mismatch"
chapter 14 testing_for_exceptional_conditions_in_unit_tests.test: WASM_EXECUTION_FAILURE. "Exception: indirect call type mismatch"


100%|██████████| 14/14 [01:01<00:00,  4.40s/it]

✨ There are 1 different callstacks.
[['1: method_vectorcall_NOARGS',
  '2: ../../Objects/call.c',
  '3: _PyEval_EvalFrameDefault',
  '4: ../../Python/ceval.c',
  '5: _PyFunction_Vectorcall']]


First trial:

It seems all(*16*) exceptions are in 4 kinds:

- getset_get() <- _PyObject_GenericGetAttrWithDict() <- PyObject_GenericGetAttr() <- PyObject_GetAttr().
- getset_set()
- PyUnicode_FromFormatV()
- PyNumber_InPlaceAdd()


Second trail:

It seems all(*2*) exceptions are in 1 kinds
- method_vectorcall_NOARGS <- _PyEval_EvalFrameDefault <- _PyFunction_Vectorcall

##### callstack A.

Refer to [this one](./cookbook/src/9/defining_context_managers_the_easy_way/example2.dtl.ctk)


```
$ head -n 18 ./cookbook/src/9/defining_context_managers_the_easy_way/example2.dtl.ctk

1: getset_get
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/descrobject.c:202:16
2: _PyObject_GenericGetAttrWithDict
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/object.c:1278:19
3: PyObject_GenericGetAttr
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/object.c:1368:5
4: PyObject_GetAttr
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/object.c:916:18
5: builtin_getattr
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Python/bltinmodule.c:1134:18
6: _PyEval_EvalFrameDefault
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Python/ceval.c:5050:29
7: ../../Python/ceval.c
	at unknown:73:16
8: _PyFunction_Vectorcall
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/call.c:398:1
9: _PyVectorcall_Call
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/call.c:257:24
````


Observation in .wasm.

- `type[0] (i32) -> i32`
- `type[2] (i32, i32) -> i32`
- `type[3] (i32, i32, i32) -> i32`
- `func[570] sig=3 <getset_get>`
- `func[8785] sig=0 <Nuitka_Function_get_module>`
- ` - elem[246] = ref.func:570 <getset_get>`
- ` - elem[4628] = ref.func:8769 <Nuitka_Function_get_qualname>`
- ` - elem[4630] = ref.func:8771 <Nuitka_Function_get_name>`
- ` - elem[4634] = ref.func:8775 <Nuitka_Function_get_dict>`
- ` - elem[4632] = ref.func:8773 <Nuitka_Function_get_doc>`
- ` - elem[4644] = ref.func:8785 <Nuitak_Function_get_module>`
- ` - elem[4648] = ref.func:8789 <Nuitka_Function_get_annotations>`
- ``` wasm
   039e3f func[570] <getset_get>:
     ;; ...
     039f29: 20 01                      |     local.get 1
     039f2b: 20 04                      |     local.get 4
     039f2d: 28 02 10                   |     i32.load 2 16
     039f30: 20 05                      |     local.get 5               ;; <|-- elem idx 4644 -> Nuitka_Function_get_module
     039f32: 11 82 80 80 80 00 00       |     call_indirect 0 (type 2)  ;; <|-- Exception: indirect call type mismatch
     039f39: 21 04                      |     local.set 4
     039f3b: 0c 01                      |     br 1
  ```

Observation:

- ➡️ `_PyObject_GenericGetAttrWithDict()`. https://github.com/python/cpython/blob/3.11/Objects/object.c#L1243
- `f = Py_TYPE(descr)->tp_descr_get;`
- `res = f(descr, obj, (PyObject *)Py_TYPE(obj));`. https://github.com/python/cpython/blob/3.11/Objects/object.c#L1278
- ➡️ `getset_get()`. https://github.com/python/cpython/blob/3.11/Objects/descrobject.c#L193
- `descr->d_getset->get(obj, descr->d_getset->closure)`
- `return descr_get_trampoline_call(descr->d_getset->get, obj, descr->d_getset->closure);`. https://github.com/python/cpython/blob/3.11/Objects/descrobject.c#L202
- ➡️ `descr_get_trampoline_call()`
  ``` c
  // see pycore_object.h
  #if defined(__EMSCRIPTEN__) && defined(PY_CALL_TRAMPOLINE)
  #include <emscripten.h>
  EM_JS(int, descr_set_trampoline_call, (setter set, PyObject *obj, PyObject *value, void *closure), {
      return wasmTable.get(set)(obj, value, closure);
  });
  
  EM_JS(PyObject*, descr_get_trampoline_call, (getter get, PyObject *obj, void *closure), {
      return wasmTable.get(get)(obj, closure);
  });
  #else
  #define descr_set_trampoline_call(set, obj, value, closure) \
      (set)((obj), (value), (closure))
  
  #define descr_get_trampoline_call(get, obj, closure) \
      (get)((obj), (closure))    // <|-- HERE!
  
  #endif // __EMSCRIPTEN__ && PY_CALL_TRAMPOLINE
  ```
- ***indirect call type mismatch***. `func(i32,i32)->i32`(in bytecode) != `func(i32)->i32`(on stack). 

---

More observation:
- *Objects/object.c* for the built-in methods
- [`builtin_getattr()` -> `getattr()`](https://docs.python.org/3/library/functions.html#getattr)
- > For example, `getattr(x, 'foobar')` is equivalent to `x.foobar`

--- 
``` c
// in Include/cpython/object.h

typedef PyObject *(*descrgetfunc) (PyObject *, PyObject *, PyObject *);

struct _typeobject {
  //...
  descrgetfunc tp_descr_get;
  //...
}; 
```

use lldb to track values of `f` from `f = Py_TYPE(descr)->tp_descr_get`, and `name->ob_type->tp_name` in `_PyObject_GenericGetAttrWithDict()`

- `method_get()` for `str`
- `member_get()` for `str`
- `getset_get()` for `str`
- `property_descr_get()` for `str`
- `func_descr_get()` for `str`
- `wrapperdescr_get()` for `str`

⬆️ matches definitions in *Objects/descrobject.c*


###### About `Nuitak_Function_get_module()`

- it is a function defined in *__py2wasm/example2.build/static_src/CompiledFunctionType.c*
- it is involved by 
  ``` c
  // ./nuitka/wasi-python/include/python3.11/descrobject.h
  typedef PyObject *(*getter)(PyObject *, void *);

  // descrobject.h <- Python.h <- prelude.h

  // ./nuitka/build/static_src/CompiledFunctionType.c
  #include "nuitka/prelude.h"

  static PyObject *Nuitka_Function_get_module(struct Nuitka_FunctionObject *function) { 
    //... 
  }

  static PyGetSetDef Nuitka_Function_getset[] = {
    //...
    {(char *)"__module__", (getter)Nuitka_Function_get_module, (setter)Nuitka_Function_set_module, NULL}
    //...
  }; 

  PyTypeObject Nuitka_Function_Type = {
    //...
    Nuitka_Function_getset, // tp_getset
    //...
  }
  ```

First 💥 is here

Cast `Nuitka_Function_get_module()` from `PyObject*(struct Nuitka_FunctionObject*)` to `PyObject*(PyObject*, void*)`. 

Similar (unsafe) cast is found in *./build/static_src/CompiledGeneratorType.c*, *./build/static_src/CompiledAsyncgenType.c*, *./build/static_src/MetaPathBasedLoaderResourceReaderFiles.c*, *./build/static_src/CompiledCoroutineType.c*, *./build/static_src/CompiledFrameType.c*, *./build/static_src/MetaPathBasedLoader.c*, *./build/static_src/CompiledFunctionType.c*, *./build/static_src/CompiledMethodType.c* and *./build/static_src/CompiledCellType.c*. 53 in all.

a suggestion patch is similar with:

``` patch
@@ -78,7 +78,11 @@ static long Nuitka_Function_tp_hash(struct Nuitka_FunctionObject *function) {
     return function->m_counter;
 }
 
+#if __wasi__
+static PyObject *Nuitka_Function_get_name(struct Nuitka_FunctionObject *function, void *data) {
+#else
 static PyObject *Nuitka_Function_get_name(struct Nuitka_FunctionObject *function) {
+#endif
     CHECK_OBJECT((PyObject *)function);
     assert(Nuitka_Function_Check((PyObject *)function));
     assert(_PyObject_GC_IS_TRACKED(function));
```

##### callstack B.

A call stack with more detail

```
1: method_vectorcall_NOARGS
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/descrobject.c:453:24
2: ../../Objects/call.c
	at unknown:92:11
3: _PyEval_EvalFrameDefault
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Python/ceval.c:4769:23
4: ../../Python/ceval.c
	at unknown:73:16
5: _PyFunction_Vectorcall
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/call.c:398:1
6: ../../Objects/classobject.c
	at unknown:0:11
7: ../../Objects/call.c
	at unknown:92:11
8: _PyEval_EvalFrameDefault
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Python/ceval.c:4453:17
9: ../../Python/ceval.c
	at unknown:73:16
10: _PyFunction_Vectorcall
	at /Users/syrusakbary/Development/cpython-3.11/builddir/wasi/../../Objects/call.c:398:1
11: ../../Objects/classobject.c
	at unknown:0:11
12: CALL_FUNCTION_WITH_ARGS3
	at unknown:?:?
13: impl___main__$$$function__2_test_bad_int
	at unknown:?:?
```

**OBSERVATIONS**

- in *cpython/Objects/descrobject.c*
  ``` c
  static PyObject *
  method_vectorcall_NOARGS(
      PyObject *func, PyObject *const *args, size_t nargsf, PyObject *kwnames) {
    PyThreadState *tstate = _PyThreadState_GET();

    PyCFunction meth = (PyCFunction)method_enter_call(tstate, func);

    PyObject *result = _PyCFunction_TrampolineCall(meth, args[0], NULL);   // <|-- HERE!
  }
  ```
- in *cpython/Include/internal/pycore_object.h*
  ``` c
  #if defined(__EMSCRIPTEN__) && defined(PY_CALL_TRAMPOLINE)
  #define _PyCFunction_TrampolineCall(meth, self, args) \
      _PyCFunctionWithKeywords_TrampolineCall( \
          (*(PyCFunctionWithKeywords)(void(*)(void))meth), self, args, NULL)
  extern PyObject* _PyCFunctionWithKeywords_TrampolineCall(
      PyCFunctionWithKeywords meth, PyObject *, PyObject *, PyObject *);
  #else
  #define _PyCFunction_TrampolineCall(meth, self, args) \
      (meth)((self), (args))  // <| -- HERE!
  #define _PyCFunctionWithKeywords_TrampolineCall(meth, self, args, kw) \
      (meth)((self), (args), (kw))
  #endif // __EMSCRIPTEN__ && PY_CALL_TRAMPOLINE
  ```
- in *cpython/Objects/descrobject.c*. it is a `ml_meth`
  ``` c
  static inline funcptr
  method_enter_call(PyThreadState *tstate, PyObject *func)
  {
      if (_Py_EnterRecursiveCallTstate(tstate, " while calling a Python object")) {
          return NULL;
      }
      return (funcptr)((PyMethodDescrObject *)func)->d_method->ml_meth;  
  }
  ```
- in *cpython/Include/methodobject.h*. `ml_meth` is a member of `PyMethodDef`
  ``` c
  typedef PyObject *(*PyCFunction)(PyObject *, PyObject *);

  typedef struct {
      const char *ml_name;
      PyCFunction ml_meth;
      int ml_flags;
      const char *ml_doc;
  } PyMethodDef;
  ```
- `PyCFunction` should be `PyObject *(*)(PyObject *, PyObject *)`.
- in *py2wasm/nuitka/build/static_src/CompiledAsyncgenType.c*, there are some unsafe function pointer casting



#### out of bounds memory access

In [12]:
summarize_callstacks(
    chapter_x_wasm_exec_failed,
    "Exception: out of bounds memory access", 
    "out_of_bounds_memory_access_call_stack.pkl"
)

  0%|          | 0/14 [00:00<?, ?it/s]

chapter 10 loading_modules_from_a_remote_machine_using_import_hooks.urlimport: WASM_EXECUTION_FAILURE. "./urlimport.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
Traceback (most recent call last):
  File "./urlimport.py", line 6, in <module>
    from urllib.request import urlopen
    
  File "./urllib/request.py", line 88, in <module>
  File "./http/client.py", line 72, in <module>
ModuleNotFoundError: No module named 'email.message'

Exception: out of bounds memory access"


 71%|███████▏  | 10/14 [00:37<00:15,  3.77s/it]

chapter 11 interacting_with_http_services_as_a_client.example2: WASM_EXECUTION_FAILURE. "Exception: out of bounds memory access"
chapter 11 interacting_with_http_services_as_a_client.example1: WASM_EXECUTION_FAILURE. "Exception: out of bounds memory access"
chapter 11 interacting_with_http_services_as_a_client.example3: WASM_EXECUTION_FAILURE. "Exception: out of bounds memory access"


100%|██████████| 14/14 [02:01<00:00,  8.65s/it]

✨ There are 3 different callstacks.
[['1: visit_decref',
  '2: func_traverse',
  '3: ../../Modules/gcmodule.c',
  '4: ../../Modules/gcmodule.c',
  '5: Py_FinalizeEx'],
 ['1: loadTriggeredModule',
  '2: _EXECUTE_EMBEDDED_MODULE',
  '3: _nuitka_loader_exec_module',
  '4: cfunction_call',
  '5: _PyObject_MakeTpCall'],
 ['1: strcmp',
  '2: loadTriggeredModule',
  '3: _EXECUTE_EMBEDDED_MODULE',
  '4: _nuitka_loader_exec_module',
  '5: cfunction_call']]


##### callstack A

> enable `--debug` of nuitka command options, enable assert

```
Assertion failed: current (/home/vscode/.local/lib/python3.11/site-packages/py2wasm-2.5rc1-py3.11.egg/nuitka/build/static_src/MetaPathBasedLoader.c: findEntry: 251)
```

In [1]:
! head -n 10 cookbook/src/11/interacting_with_http_services_as_a_client/example1.dbg.ctk

#00: 0x34106c - abort
#01: 0x349f6c - __assert_fail
#02: 0x39497a - loadTriggeredModule
#03: 0x3945f4 - _EXECUTE_EMBEDDED_MODULE
#04: 0x3983aa - _nuitka_loader_exec_module
#05: 0x7cb2d - cfunction_call
#06: 0x2e3d1 - _PyObject_MakeTpCall
#07: 0x2eafe - PyObject_Vectorcall
#08: 0x126749 - _PyEval_EvalFrameDefault
#09: 0x12858c - _PyEval_Vector


- assertion
  ``` c
  static struct Nuitka_MetaPathBasedLoaderEntry *findEntry(char const *name) {
      struct Nuitka_MetaPathBasedLoaderEntry *current = loader_entries;
      assert(current);   // <|-- HERE
  ```
- it is about
  ```c
  static struct Nuitka_MetaPathBasedLoaderEntry *loader_entries = NULL;
  ```
- from
  ``` c
  static void loadTriggeredModule(PyThreadState *tstate, char const *name, char const *trigger_name) {
      char trigger_module_name[2048];
  
      copyStringSafe(trigger_module_name, name, sizeof(trigger_module_name));
      appendStringSafe(trigger_module_name, trigger_name, sizeof(trigger_module_name));
  
      struct Nuitka_MetaPathBasedLoaderEntry *entry = findEntry(trigger_module_name);     // <|-- HERE
  ```
- after loading modules. `loader_entries` was set to `NULL`.
  ``` log
  assign loader_entries with 0x99b9c0
  loadTriggeredModule(0x8f3a88, inspect, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, ast, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, contextlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, os, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, stat, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, stat, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _collections_abc, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _collections_abc, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, posixpath, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, genericpath, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, genericpath, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, posixpath, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, os, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, keyword, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, keyword, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, operator, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, operator, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, reprlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, reprlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, functools, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, types, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, types, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, functools, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, contextlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, enum, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, enum, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, ast, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, dis, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, opcode, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, opcode, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, dis, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections.abc, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections.abc, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, importlib.machinery, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, importlib.machinery, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, linecache, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, tokenize, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._compiler, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._parser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._constants, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._constants, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._parser, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._casefix, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._casefix, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._compiler, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, copyreg, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, copyreg, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, token, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, token, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, tokenize, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, linecache, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, inspect, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, __main__, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib.request, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, base64, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, struct, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, struct, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, base64, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, bisect, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, bisect, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, hashlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http.client, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.parser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.feedparser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.errors, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.errors, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email._policybase, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.header, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.quoprimime, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, string, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, string, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.quoprimime, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.base64mime, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.base64mime, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.charset, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.encoders, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, quopri, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, quopri, -postLoad)... loader_entries=0assign loader_entries with 0x99b9c0
  loadTriggeredModule(0x8f3a88, inspect, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, ast, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, contextlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, os, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, stat, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, stat, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _collections_abc, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _collections_abc, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, posixpath, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, genericpath, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, genericpath, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, posixpath, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, os, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, keyword, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, keyword, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, operator, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, operator, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, reprlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, reprlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, functools, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, types, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, types, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, functools, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, contextlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, enum, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, enum, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, ast, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, dis, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, opcode, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, opcode, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, dis, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections.abc, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, collections.abc, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, importlib.machinery, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, importlib.machinery, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, linecache, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, tokenize, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._compiler, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._parser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._constants, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._constants, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._parser, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._casefix, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._casefix, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re._compiler, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, copyreg, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, copyreg, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, re, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, token, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, token, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, tokenize, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, linecache, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, inspect, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, __main__, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, urllib.request, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, base64, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, struct, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, struct, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, base64, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, bisect, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, bisect, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, _hashlib, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, hashlib, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, http.client, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.parser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.feedparser, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.errors, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.errors, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email._policybase, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.header, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.quoprimime, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, string, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, string, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.quoprimime, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.base64mime, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.base64mime, -postLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.charset, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, email.encoders, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, quopri, -preLoad)... loader_entries=0x99b9c0
  loadTriggeredModule(0x8f3a88, quopri, -postLoad)... loader_entries=0    💥
  ```


# Py2wasm

download [py2wasm](https://files.pythonhosted.org/packages/db/42/bfd54e548db0f317a5e4366924c21563666929a8087e7428f4e966663a19/py2wasm-2.6.3.tar.gz) or clone *https://github.com/wasmerio/py2wasm* and checkout *wasi* branch

> https://github.com/wasmerio/py2wasm/tree/wasi

in [setup.py](./py2wasm-2.6.3/setup.py), found `"py2wasm = nuitka.__main__:py2wasm"`.

in [py2wasm](/home/vscode/.local/bin/py2wasm). found `from nuitka.__main__ import py2wasm`.

in [nuitka/__main__.py](./py2wasm-2.6.3/nuitka/__main__.py), found `py2wasm()`.

``` python
def py2wasm():
    from nuitka.utils.wasi_sdk import try_get_sdk_path

    sdk_path = try_get_sdk_path()
    # WASI_SDK_DIR = os.environ.get("WASI_SDK_DIR")
    # if not WASI_SDK_DIR:
    #     print("Please set the WASI_SDK_DIR to continue")
    #     return -1
    clang_path = "%s/bin/clang" % sdk_path
    if not os.path.isfile(clang_path):
        print("py2wasm: The SDK clang file doesn't exist: %s" % clang_path)
        return -1
    os.environ["CC"] = clang_path

    ######################################## 

    import argparse

    parser = argparse.ArgumentParser(description='py2wasm is a program to compile Python to WebAssembly', prog="py2wasm")
    parser.add_argument('filename', help='The python file to compile')
    parser.add_argument('-o', '--output', help='The output wasm')
    args = parser.parse_args(sys.argv[1:])

    extra_args = []

    output_dir = os.path.join(os.path.dirname(__file__), "__py2wasm")
    output_filename = "output.wasm"
    extra_args = [
        "--output-dir=%s" % output_dir,
        "--output-filename=%s" % output_filename,
        "--remove-output"
    ]

    binary_name = "nuitka"
    if "NUITKA_BINARY_NAME" in os.environ:
        binary_name = os.environ["NUITKA_BINARY_NAME"]

    sys.argv = [
        binary_name,
        args.filename,
        "--standalone",
        "--static-libpython=yes",
        "--disable-ccache",
        # "--onefile",
        "--lto=yes",
    ] + extra_args

    final_output_file = args.output or args.filename.replace(".py", ".wasm")
    os.environ["PY2WASM_OUTPUT"] = final_output_file

    main()
```

Obviously, it uses `nuitka` to compile python scripts, use wasi-sdk as backends, to an executable binary(generate example2.bin). Just like:

``` bash
$ nuitka --output-dir=$(pwd)__py2wasm --output-filename=output.wasm --remove-output --standalone --static-libpython=yes --disable-ccache --lto=yes example2.py
```

It involves [a pre-built wasi-python](./py2wasm-2.6.3/nuitka/wasi-python/). ✨ Maybe we can replace it with latest build or our own build

*nuitka/PythonVersions.py* involve ⬆️*wasi-python* via `getSystemPrefixPath()`

``` python
    # WASI-Python
    wasi_python_dir = os.path.join(os.path.dirname(__file__), "wasi-python")
    _the_sys_prefix = wasi_python_dir
    return _the_sys_prefix
```

## py2wasm dig deep

### main entrance (py2wasm)

patch */home/vscode/.local/lib/python3.11/site-packages/nuitka/__main__.py*

``` python
    extra_args = [
        "--output-dir=%s" % output_dir,
        "--output-filename=%s" % output_filename,
        # "--remove-output"
    ]

    sys.argv = [
        binary_name,
        args.filename,
        "--standalone",
        "--static-libpython=yes",
        "--disable-ccache",
        # "--onefile",
        "--lto=yes",
        "--debug",
        "--verbose",
    ] + extra_args
```

### py2wasm log and generation

translate `example2.py` to `example2.tmp.wasm` and analyze the log

``` bash
$ pwd
/workspaces/wamr-with-py/cookbook/src/9/defining_context_managers_the_easy_way
```

``` bash
$ py2wasm -o example2.tmp.wasm ./example2.py > ./example2.py2wasm.log
Nuitka-Options: Used command line options: ./example2.py --standalone --static-libpython=yes --disable-ccache --lto=yes --debug --verbose --output-dir=/home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm --output-filename=output.wasm
Nuitka: Starting Python compilation with Nuitka '2.6.3' on Python '3.11' commercial grade 'not installed'.
Nuitka-Plugins:anti-bloat: Not including '_bisect' automatically in order to avoid bloat, but this may cause: may slow down by using fallback implementation.
Nuitka-Plugins:anti-bloat: Not including 'socket' automatically in order to avoid bloat, but this may cause: can break calls of 'email.utils.make_msgid()'.
Nuitka: Completed Python level compilation and optimization.
Nuitka: Generating source code for C backend compiler.
Nuitka: Running data composer tool for optimal constant value handling.
Nuitka: Running C compilation via Scons.
Nuitka-Scons: Backend C compiler: ~/.local/lib/python3.11/site-packages/nuitka/wasi-sdk/21/sdk-Linux/bin/clang (clang 17.0.6).
Nuitka-Scons: Backend linking program with 7 files (no progress information available for this stage).
wasm-ld: warning: unknown -z value: noexecstack
Nuitka: Keeping build directory '/home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.build'.    => ✨
Nuitka: Successfully created '/home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.dist/output.wasm'.  => ⭐
```

There are two directories generated during the process:

``` bash
$ ls /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/
example2.build  example2.dist

✨
$ ls /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.build
@link_input.txt   __constants.bin  __constants.const  __constants.o    __constants_data.c  __helpers.c  __helpers.o  __loader.o           module.__main__.c      module.__main__.o       scons-report.txt
__bytecode.const  __constants.c    __constants.h      __constants.txt  __constants_data.o  __helpers.h  __loader.c   build_definitions.h  module.__main__.const  scons-error-report.txt  static_src

$ ls /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.build/static_src/
CompiledFunctionType.c  CompiledFunctionType.o  MainProgram.c  MainProgram.o

$ file module.__main__.o
module.__main__.o: LLVM IR bitcode

$ cat /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.build/\@link_input.txt
"./__constants.o"
"./__constants_data.o"
"./__helpers.o"
"./__loader.o"
"./module.__main__.o"
"./static_src/MainProgram.o"
"./static_src/CompiledFunctionType.o"

⭐
$ ls /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.dist/
_blake2.so     _codecs_hk.so       _codecs_kr.so    _csv.so       _hashlib.so  _md5.so             _pickle.so           _sha1.so    _sha512.so      _typing.so   grp.so          liblzma.so.5  select.so
_bz2.so        _codecs_iso2022.so  _codecs_tw.so    _datetime.so  _heapq.so    _multibytecodec.so  _posixsubprocess.so  _sha256.so  _statistics.so  binascii.so  libbz2.so.1.0   math.so       unicodedata.so
_codecs_cn.so  _codecs_jp.so       _contextvars.so  _decimal.so   _lzma.so     _opcode.so          _random.so           _sha3.so    _struct.so      fcntl.so     libcrypto.so.3  output.wasm   zlib.so

$ file /home/vscode/.local/lib/python3.11/site-packages/nuitka/__py2wasm/example2.dist/_csv.so
_csv.so: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, BuildID[sha1]=94bd19397a73efbca936587952254e315165d7a1, with debug_info, not stripped
```

---
Observations:

- the content of `example2.py` goes to `module.__main__.c`

### Enable trace mode

`--trace-execution` for nuitka command line options

### Enable Verbose in *MetaPathBasedLoader*

- `isVerbose()` in *nuitka/build/static_src/MetaPathBasedLoader.c* \
      <- `Py_VerboseFlag` in *nuitka/build/static_src/MainProgram.c* \
      <- `SYSFLAG_VERBOSE` in *${OUTPUT}/example.build/build_definitions.h* (GENERATED) `--remove-output` will delete it \
      <- `python_sysflag_verbose` in *nuitka/MainControl.py* \
      <- `Options.hasPythonFlagTraceImports()` in *nuitka/Options.py* \
      <- `nuitka --python-flags=trace_imports`

### the py2wasm patch

- *nuitka/wasi-python/*. from cpython wasi building

## ~~py2wasm install Nuitka~~

@ *~/.local/lib/python3.11/site-packages/nuitka/*

## Install local py2wasm

``` bash
$ pwd
<py2wasm>/repo

$ python3 setup.py --verbose install --user

$ which py2wasm
<home>/.local/bin/py2wasm

$ py2wasm --version
...
py2wasm: error: the following arguments are required: filename
```

# CPython

> https://devguide.python.org/getting-started/setup-building/#wasi

There is a wasi-python directory in the py2wasm installation.

``` bash
$ ls /home/vscode/.local/lib/python3.11/site-packages/nuitka/wasi-python/
bin  include  lib
```

## cross-compile to wasm32-wasi

```bash
$ pwd
<cpython-repo-root>

$ git switch v3.11

$ make clean
$ rm python

$ ./Tools/wasm/wasm_build.py wasi build
...

$ cd builddir/wasi
# install to /a/b/c
$ make prefix=/a/b/c install

$ cp builddir/build/Modules/expat/libexpat.a /a/b/c/lib
```

---


```
wasm-ld: warning: unknown -z value: noexecstack
wasm-ld: error: unable to find library -lmpdec
wasm-ld: error: unable to find library -lexpat
clang: error: linker command failed with exit code 1 (use -v to see invocation)
```


# Nuitka

Use `python3 -m nuitka` to compile python scripts to an executable binary(generate example2.bin).

`python3 -m nuitka --debug --verbose --standalone --static-libpython=yes --lto=yes <script.py>`

```
Nuitka: Running C compilation via Scons.
Nuitka-Scons: Backend C compiler: gcc (gcc 12).
Nuitka-Scons: Backend linking program with 7 files (no progress information available for this stage).
Nuitka-Scons:WARNING: You are not using ccache, re-compilation of identical code will be slower than necessary. Use your OS package manager to
Nuitka-Scons:WARNING: install it.
Nuitka-Postprocessing: Creating single file from dist folder, this may take a while.
Nuitka-Onefile: Running bootstrap binary compilation via Scons.
Nuitka-Scons: Onefile C compiler: gcc (gcc 12).
Nuitka-Scons: Onefile linking program with 1 files (no progress information available for this stage).
Nuitka-Scons:WARNING: You are not using ccache, re-compilation of identical code will be slower than necessary. Use your OS package manager to
Nuitka-Scons:WARNING: install it.
Nuitka-Onefile: Using compression for onefile payload.
Nuitka-Onefile: Onefile payload compression ratio (29.13%) size 40944778 to 11926675.
Nuitka-Onefile: Keeping onefile build directory 'example2.onefile-build'.
Nuitka: Keeping dist folder 'example2.dist' for inspection, no need to use it.
Nuitka: Keeping build directory 'example2.b
```

It generates a binary file depends on minimum 

```
        linux-vdso.so.1 (0x00007ffd98f5d000)
        libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fea0c6fe000)
        /lib64/ld-linux-x86-64.so.2 (0x00007fea0c919000)
```

There are 3 directories: *$script.build*, *$script.dist*, *$script.onefile-build*.




check callstack with lldb

``` lldb
* thread #1, name = 'example2.bin', stop reason = breakpoint 2.1
  * frame #0: 0x00007ffff7bc42f9 libpython3.11.so.1.0`getset_get at descrobject.c:194:1
    frame #1: 0x00007ffff7bbfa95 libpython3.11.so.1.0`_PyObject_GenericGetAttrWithDict at object.c:1278:19
    frame #2: 0x00007ffff7bbbe4c libpython3.11.so.1.0`_PyObject_LookupAttr at object.c:951:19
    frame #3: 0x00007ffff7bfe6d1 libpython3.11.so.1.0`builtin_hasattr at bltinmodule.c:1186:9
    frame #4: 0x00007ffff7bfe683 libpython3.11.so.1.0`cfunction_vectorcall_FASTCALL at methodobject.c:427:24
    frame #5: 0x00007ffff7bbb613 libpython3.11.so.1.0`PyObject_Vectorcall at pycore_call.h:92:11
    frame #6: 0x00007ffff7bbb5f7 libpython3.11.so.1.0`PyObject_Vectorcall(callable=0x00007ffff74110d0, args=<unavailable>, nargsf=<unavailable>, kwnames=<unavailable>) at call.c:299:12
    frame #7: 0x00007ffff7bb079b libpython3.11.so.1.0`_PyEval_EvalFrameDefault at ceval.c:4769:23
    frame #8: 0x00007ffff7baf32a libpython3.11.so.1.0`_PyEval_Vector at pycore_ceval.h:73:16
    frame #9: 0x00007ffff7bfd713 libpython3.11.so.1.0`builtin___build_class__ at bltinmodule.c:201:12
    frame #10: 0x00007ffff7bfd568 libpython3.11.so.1.0`cfunction_vectorcall_FASTCALL_KEYWORDS at methodobject.c:443:24
    frame #11: 0x00007ffff7bbb613 libpython3.11.so.1.0`PyObject_Vectorcall at pycore_call.h:92:11
    frame #12: 0x00007ffff7bbb5f7 libpython3.11.so.1.0`PyObject_Vectorcall(callable=0x00007ffff7410ae0, args=<unavailable>, nargsf=<unavailable>, kwnames=<unavailable>) at call.c:299:12
    frame #13: 0x00007ffff7bb079b libpython3.11.so.1.0`_PyEval_EvalFrameDefault at ceval.c:4769:23
    frame #14: 0x00007ffff7baf32a libpython3.11.so.1.0`_PyEval_Vector at pycore_ceval.h:73:16
    frame #15: 0x00007ffff7bfcfb8 libpython3.11.so.1.0`PyEval_EvalCode at ceval.c:1148:21
    frame #16: 0x00007ffff7bfcee5 libpython3.11.so.1.0`exec_code_in_module at import.c:764:9
    frame #17: 0x00007ffff7bfc4d5 libpython3.11.so.1.0`PyImport_ImportFrozenModuleObject at import.c:1407:9
    frame #18: 0x00007ffff7bfc41c libpython3.11.so.1.0`PyImport_ImportFrozenModule at import.c:1447:11
    frame #19: 0x00007ffff7bf034d libpython3.11.so.1.0`pycore_interp_init at pylifecycle.c:187:9
    frame #20: 0x00007ffff7beb6f2 libpython3.11.so.1.0`pyinit_core.constprop.0 at pylifecycle.c:901:14
    frame #21: 0x00007ffff7beb486 libpython3.11.so.1.0`Py_InitializeFromConfig at pylifecycle.c:1254:14
    frame #22: 0x0000555555563465 example2.bin`Nuitka_Py_Initialize at MainProgram.c:991:14
    frame #23: 0x000055555556385e example2.bin`main(argc=1, argv=0x00007fffffffdbd8) at MainProgram.c:1346:5
    frame #24: 0x00007ffff77f924a libc.so.6`___lldb_unnamed_symbol3156 + 122
    frame #25: 0x00007ffff77f9305 libc.so.6`__libc_start_main + 133
    frame #26: 0x000055555555e071 example2.bin`_start + 33
```